##### 분급→선급 수수료 정산 프로그램 : 본 프로그램 사용시 파일명,회사명 확인 후 실행바람
  1. 분급,선급원장 읽기 : 첫행은 반드시 컬럼명이어야  
  2. 분급원장 전처리 : glad 계약 추출, 문홍주건 정제, 지점코드 세팅  
  3. 선급원장 전처리: 해당회사,장기
  4. 증번,회차별 sort  :<span style="color:yellow">미사용</span>
  5. 선급,분급 차이 산출 

In [1]:
# import sys
# sys.path.insert(0, 'd:/MyPython/')
# from mypkgs import GladUtil as glad
import pandas as pd
import GladUtil as glad
import time
import os
pd.set_option('mode.chained_assignment',  None) # <==== 경고를 끈다

# 제이엔제이에서 글래드프라임으로 이동건은 지점명을 제이엔제이로 변경-20210701 "
def glad_prime(df_p):
    cond = " 팀 == '프라임2팀' "
    temp = [ '글래드제이엔제이' if x < '20210701' else '글래드프라임' for x in df_p.query(cond)['계약일자'] ]
    cond = df_p['팀'] == '프라임2팀'
    df_p.loc[cond ,'지점'] = temp
    return df_p

#라온 사원중 복귀자-mp와 정산안함
def glad_return(dfT):
    # dfT1 = dfT[~((dfT.사원명.isin(['박종철','이용진','박종호'])) & (dfT.계약일자 >= '20220801'))] 
    dfT1 = dfT[~( (dfT.지점 == '글래드라온')&(dfT.계약일자 >= '20220801' ))]
    return dfT1

def read_source_div(df_div):
    '''분급원장 read'''
    # df_div = pd.DataFrame()
    # df_div = pd.read_excel(inFile1, sheet_name=company,header=0,dtype={'증권번호':object})
    
    df_div = df_div[df_div.총괄=='GLAD']
    df_div['계약일자'] = df_div.계약일자.map(lambda x: str(x)[:8])
    # df_div = df_div[~((df_div.사원명=='박종철') & (df_div.계약일자 >= '20220801'))]  #박종철 22.8월이후 MP와 정산X
    df_div = glad_return(df_div)
    df_div = glad.BrhConv(df_div,'지점그룹')                               #글래드직속(구,메트로)로지점분리, 직영/지사구분
    df_div = glad_prime(df_div)
    df_div['보험료'].fillna(0, inplace=True)  #결측치 보정
    df_div['수수료'].fillna(0, inplace=True)  #결측치 보정
    return df_div

def read_source_adv(dfW):
    ''' 선급원장 read -- 최초1회만'''
    #선급 원장: 기존 방식의 계산값(글로벌 -> MP)
    start = time.time()  # 시작 시간 저장
    try:
        if len(dfW)==0:
            raise NameError
        val = str(dfW.head(1)['지급년월'][0])
        if val == ym:
            raise NameError("원장년월 변경필요")
    # catch when df1 is None
    except NameError:    
        print('선급원장 start reading...')
        dfW = pd.read_excel(inFile2, header=0, sheet_name='전체',dtype={'계약일자':object})  
    finally:
    #SettingWithCopyWarning 방지를 위해 deep copy사용(https://emilkwak.github.io/pandas-dataframe-settingwithcopywarning)
        df_advance = dfW[(dfW.보험사==company)&(dfW.계약종류=='장기')].copy() 
        # df_advance = df_advance[~((df_advance.사원명=='박종철') & (df_advance.계약일자 >= '20220801'))]  #박종철 22.8월이후 MP와 정산X
        df_advance = glad_return(df_advance)
        df_advance = glad_prime(df_advance)
        df_advance['보험료'].fillna(0, inplace=True)  #결측치 보정
        df_advance['수수료계'].fillna(0, inplace=True)  #결측치 보정
        df_advance = df_advance.sort_values(by=['증권번호','납입회차','상태','보험료']).reset_index()
        print('선급원장 read completion ')
    print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    df_advance = glad.BrhConv(df_advance,'지점그룹')  
    return dfW, df_advance

def make_outfile():
    ''' 회사별 시트 생성'''
    dfT = pd.DataFrame(df_div.groupby('지점')[['수수료']].agg('sum'))

#     df_advance = glad.BrhConv(df_advance,'지점그룹')   
    dfT1 = pd.DataFrame(df_advance.groupby('지점')[['수수료계']].agg('sum'))
    dfT = pd.concat([dfT,dfT1],axis=1)
    dfT.reset_index(inplace=True)

    dfT.rename(columns = {'수수료':'분급수수료','수수료계':'선급수수료'}, inplace=True)
    dfT['분급수수료'].fillna(0, inplace=True)  #결측치 보정
    dfT['선급수수료'].fillna(0, inplace=True)  #결측치 보정
    dfT['차액'] = dfT.분급수수료 - dfT.선급수수료
    차액합 = dfT['차액'].sum()

    # row합계
    dfT.loc[len(dfT)]= dfT[['분급수수료','선급수수료','차액']].sum()
    dfT['지점'].fillna('합계',inplace=True)

    dfT['ym'] = ym
    dfT['회사'] = company
    dfT = dfT[['ym','회사','지점','분급수수료','선급수수료','차액']]

    #del dfT,s1
    print('{} 선급건수 = {:,.0f} 분급건수 = {:,.0f}'.format(company, len(df_advance),len(df_div)))
    print('선급수수료-원장 = {:,.0f}'.format(df_advance.수수료계.sum() ))
    print('분급수수료-원장 = {:,.0f}'.format(df_div.수수료.sum() ) )
    print('{} 차액 = {:,.0f}'.format(company, 차액합 ) )
    return dfT

# DICT_COMP = {'롯데':'롯데손보','현대':'현대해상','KB':'KB손보','DB':'DB손보','삼성':'삼성화재'}
DIR_PATH = 'e:/글로벌백업(180329)/04.수수료작업/'

''' =========>> 당월 이자율/일수를  반드시 확인 후 수정  <<==========='''
ym ='202304'
이자율 = 10.4
일수   = 30
if ym > '202212':  #정산비율
    SHARE_RT = 20
else: SHART_RE = 30

# inf_nm1 ='분급이자/'+ ym+'_'+ comp +' 분급 원장.xlsx'
inf_nm1 = '분급이자/'+ ym + '_GLAD수입원장(분급)건별.xlsx'
inf_nm2 = ym + '수수료/'+ ym[0:4]+ ym[4:6]+ '_수입원장.xlsx'
out_nm  = ym + '수수료/'+ ym[0:4]+'-'+ym[4:6]+ ' 선급수수료이자 정산.xlsx'
# print(inf_nm1)

inFile1 = DIR_PATH + inf_nm1                   #분급원장
inFile2 = DIR_PATH + inf_nm2                   #선급원장
outFile = DIR_PATH + out_nm                    #출력파일
dfW = pd.DataFrame()
df_all = pd.DataFrame()

print('분급원장 reading')
# for comp, company in DICT_COMP.items():
df = pd.read_excel(inFile1,sheet_name =None,header=0,dtype={'증권번호':object})   
for company in df.keys():
    if len(df[company]) > 0:
        df_div = df[company]
        df_div = read_source_div(df_div)
        dfW, df_advance = read_source_adv(dfW)
        df_all = pd.concat([df_all,make_outfile()])

''' 전월 파일 열기 + 당월과 합침 '''
bef_ym = str(int(ym[:4])-1)+'12' if int(ym[4:6])-1==0 else ym[:4]+str(int(ym[4:6])-1).zfill(2)

out_nm  = '분급이자/'+ ym[0:4]+'-'+ym[4:6]+ ' 선급수수료_이자정산.xlsx'
outFile = DIR_PATH + out_nm 
befFile = outFile.replace(ym,bef_ym).replace(ym[:4]+'-'+ym[4:], bef_ym[:4]+'-'+bef_ym[4:] )

df_bef = pd.read_excel(befFile, sheet_name='월별')
df_all = pd.concat([df_bef,df_all])

''' 요약 생성 '''
df_sum = pd.pivot_table(df_all[df_all['지점']!='합계'],index=['지점'],
               values=["분급수수료","선급수수료",'차액'],
               aggfunc=sum,fill_value=0).reset_index()
df_sum['%'] = df_sum['지점'].apply(lambda x: 0.9 if x =='글래드' else (SHARE_RT if x in glad.공동지점 else 0))
df_sum['MP'] = df_sum['차액'] * df_sum['%'] / 100
df_sum['이자'] = df_sum['MP'] * ( 이자율 * 일수 / 365 ) / 100

df_sum.loc[len(df_sum)]= df_sum[['분급수수료','선급수수료','차액','MP','이자']].sum()
df_sum['지점'].fillna('합계',inplace=True)

''' to excel '''
w_mode = 'a' if os.path.isfile(outFile) else 'w'  #파일있으면 append mode
w_mode = 'w'
with pd.ExcelWriter(outFile, mode = w_mode) as writer:
    df_sum.to_excel(writer, sheet_name='요약',index=False) 
    df_all.to_excel(writer, sheet_name='월별', index=False)
    workbook = writer.book
    format1 = workbook.add_format({'num_format': '#,##0;[red]-#,##0'})
    
    worksheet = writer.sheets["요약"]
    worksheet.set_column('B:D', 14, format1)
    worksheet.set_column('F:G', 14, format1)
    worksheet.set_column('A:A',18)
    worksheet.write('H1','이자율')
    worksheet.write('H2',이자율)
    worksheet.write('I1','일수')
    worksheet.write('I2',일수)
    
    worksheet1 = writer.sheets["월별"]
    worksheet1.set_column('D:F', 14, format1)

print('<<<<<--- job completed --->>>>>')

분급원장 reading
선급원장 start reading...
선급원장 read completion 
time : 43.710004806518555
롯데손보 선급건수 = 4,369 분급건수 = 4,354
선급수수료-원장 = 48,947,203
분급수수료-원장 = 44,025,188
롯데손보 차액 = -4,922,015
선급원장 read completion 
time : 0.03899383544921875
현대해상 선급건수 = 5,025 분급건수 = 5,639
선급수수료-원장 = 51,580,364
분급수수료-원장 = 54,145,718
현대해상 차액 = 2,565,354
선급원장 read completion 
time : 0.06299877166748047
KB손보 선급건수 = 10,205 분급건수 = 10,205
선급수수료-원장 = 58,040,400
분급수수료-원장 = 69,318,141
KB손보 차액 = 11,277,741
선급원장 read completion 
time : 0.04146528244018555
DB손보 선급건수 = 5,676 분급건수 = 5,676
선급수수료-원장 = 29,152,372
분급수수료-원장 = 26,302,651
DB손보 차액 = -2,849,721
선급원장 read completion 
time : 0.03899979591369629
삼성화재 선급건수 = 4,816 분급건수 = 4,816
선급수수료-원장 = 50,147,936
분급수수료-원장 = 38,691,800
삼성화재 차액 = -11,456,136
선급원장 read completion 
time : 0.023001909255981445
MG손보 선급건수 = 1,170 분급건수 = 1,170
선급수수료-원장 = 3,111,662
분급수수료-원장 = 3,169,783
MG손보 차액 = 58,121
선급원장 read completion 
time : 0.04400014877319336
한화손보 선급건수 = 10,820 분급건수 = 10,819
선급수수료-원장 = 15,471